In [1]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [68]:
import random
rddX = sc.parallelize(random.sample(list(range(100)),100))
rddY = sc.parallelize(random.sample(list(range(100)),100))
n = float(rddX.count())
#[print(i) for i in rddX.collect()]
print(n)

100.0


In [69]:
# mean
meanX = rddX.sum()/rddX.count()
meanY = rddX.sum()/rddY.count()
print(meanX)
print(meanY)

49.5
49.5


In [70]:
# std 
stdX = (rddX.map(lambda x: (x-meanX)*(x-meanX)).sum()/n)**0.5
stdY = (rddX.map(lambda x: (x-meanY)*(x-meanY)).sum()/n)**0.5
print(stdX)
print(stdY)

28.86607004772212
28.86607004772212


In [59]:
rddXY = rddX.zip(rddY)
rddXY.take(5)

[(10, 63), (24, 82), (74, 16), (16, 64), (19, 27)]

In [71]:
# covariance
m = float(rddXY.count())
covXY = rddXY.map(lambda x_y : (x_y[0]-meanX)*(x_y[1]-meanY)).sum()/m
covXY

-43.44

In [72]:
# correlation
corrXY = covXY/(stdX*stdY)
corrXY

-0.05213321332133213

In [64]:
## Pearson corr matrix
from pyspark.mllib.stat import Statistics
col1 = sc.parallelize(range(100))
col2 = sc.parallelize(range(100,200))
col3 = sc.parallelize(range(200,100,-1))
col4 = sc.parallelize(random.sample(range(100),100))

In [66]:
data = col1.zip(col2).zip(col3).zip(col4)
data.take(5)

[(((0, 100), 200), 97),
 (((1, 101), 199), 65),
 (((2, 102), 198), 82),
 (((3, 103), 197), 5),
 (((4, 104), 196), 71)]

In [67]:
data = data.map(lambda a_b_c_d : (a_b_c_d[0][0][0],a_b_c_d[0][0][1],a_b_c_d[0][1],a_b_c_d[1]) ).map(lambda a_b_c_d : [a_b_c_d[0],a_b_c_d[1],a_b_c_d[2],a_b_c_d[3]])
print(Statistics.corr(data))

[[ 1.          1.         -1.         -0.20615662]
 [ 1.          1.         -1.         -0.20615662]
 [-1.         -1.          1.          0.20615662]
 [-0.20615662 -0.20615662  0.20615662  1.        ]]


In [81]:
# skewness
# n/((n-1)*(n-2)) is normalization term
skewnessX = n/((n-1)*(n-2)) * rddX.map(lambda x: ((x-meanX)**3)/(stdX**3) ).sum()
skewnessX

9.154591009071585e-18

In [82]:
# kurtosis
# use 1-n instead of n for division
kurtosisX = rddX.map(lambda x: ((x-meanX)**3)/(stdX**3) ).sum()/(1-n)
kurtosisX

-8.971499188890155e-18